# 🤺 Fence demo

This notebook demonstrates the use of various classes in this package. The core intent of this package is to provide a framework for interacting with language models in a way that is modular, extensible, and easy to use. The core classes are:
- `PromptTemplate`: A template for a prompt that can be rendered with a dictionary of variables (or using keywords). It wraps around jinja2.Template, but adds some additional functionality.
- `LLM`: A wrapper around a language model. 
- `Link`: Atomic LLM interaction wrapper. Takes in a PromptTemplate and a language model. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `TransformationLink`: A wrapper around a `function` that takes a dictionary of variables and returns a dictionary of variables. It can be invoked with a dictionary of variables, and will return a dictionary of variables.
- `Chain`: A collection of links that are invoked in the right order based on the input and output keys for each Link.
- `LinearChain`: A sequence of links that are invoked in the order they are passed in.

Through these classes, we can interact with LLMs in varying degrees of complexity:
1. Just call the model directly
2. Use a `PromptTemplate` for reusability and abstraction
3. Use a `Link` for atomic LLM interaction
4. Use a `Chain` to execute a collection of `Links`

Below, we'll go through each of these levels of complexity and provide examples.

**Note** This notebook assumes you have access to AWS Bedrock, as we use Bedrock's Claude-instant model to fuel our LLM interactions.

In [1]:
from pathlib import Path
current_dir = Path('.').resolve().parents[0]
import sys
sys.path.append(str(current_dir))

In [2]:
%load_ext autoreload
from fence.models.claude import ClaudeInstant
from fence.models.gpt import GPT4o
from fence.templates.string import StringTemplate
from fence.links import Link, TransformationLink
from fence.chains import Chain, LinearChain

## ⚙️ Setting up

In [3]:
# Get our model
model = GPT4o(source='demo_notebook')

### 🪨 Level 1 - Just call the damn thing

In [4]:
# Use the invoke method to call the model
model.invoke('Why is the sky blue?')

"The sky appears blue primarily due to a phenomenon called Rayleigh scattering. This scattering occurs because molecules and small particles in the Earth's atmosphere scatter short-wavelength light more effectively than long-wavelength light.\n\nHere's a more detailed breakdown of why this happens:\n\n1. **Sunlight Composition:** Sunlight, or white light, is a mixture of all visible colors, each with its own wavelength. The colors range from violet and blue (short wavelengths) to red (long wavelengths).\n\n2. **Interaction with Atmosphere:** As sunlight enters the Earth's atmosphere, it interacts with air molecules (mainly nitrogen and oxygen) and small particles.\n\n3. **Scattering:** The shorter wavelengths of light (blue and violet) are scattered in all directions by these small particles and gas molecules. This type of scattering is more effective for shorter wavelengths. \n\n4. **Human Perception:** Although violet light is scattered even more than blue light, our eyes are more se

In [5]:
# Just call the damn thing
model('Why is the sky blue?')

"The sky appears blue primarily because of a phenomenon known as Rayleigh scattering. Here's a more detailed explanation:\n\n1. **Sunlight Composition**: Sunlight, or white light, is composed of many colors, each with different wavelengths. When sunlight enters the Earth's atmosphere, it interacts with the molecules and small particles present in the air.\n\n2. **Rayleigh Scattering**: This type of scattering occurs when the particles causing the scattering are much smaller than the wavelength of the light. In the Earth's atmosphere, the gases and molecules are smaller than the wavelength of visible light. Rayleigh scattering is more effective at shorter wavelengths (the blue and violet end of the visible spectrum) than at longer wavelengths (the red end).\n\n3. **Perception of Color**: Although violet light is scattered even more than blue light, our eyes are more sensitive to blue light and less sensitive to violet light. Additionally, some of the violet light is absorbed by the uppe

### 🔨 Level 2 - Use a PromptTemplate

In [6]:
# Initialize a prompt template
prompt_template = StringTemplate('Why is the sky {color}?')
print(prompt_template)

StringTemplate: Why is the sky {color}?


In [7]:
# Render it with a dictionary
print(prompt_template.render({'color': 'blue'}))

# Render it with keyword arguments
print(prompt_template.render(color='red'))

# Input dict takes precedence over keyword arguments
print(prompt_template.render(input_dict={'color': 'blue'}, color='red'))

Why is the sky blue?
Why is the sky red?
Why is the sky red?


In [8]:
# You can concatenate prompt templates, input variables are merged
prompt_template_sky = StringTemplate('Why is the sky {color}?')
prompt_template_grass = StringTemplate('Why is the grass {color}?')
prompt_template_dress = StringTemplate('I like a dress with {pattern}.')
combined_prompt_template = prompt_template_sky + prompt_template_grass + prompt_template_dress
print(combined_prompt_template)
print(combined_prompt_template.render({'color': 'blue', 'pattern': 'polka dots'}))

StringTemplate: Why is the sky {color}? Why is the grass {color}? I like a dress with {pattern}.
Why is the sky blue? Why is the grass blue? I like a dress with polka dots.


In [9]:
# You can customize the separator
base_template = StringTemplate('Why is the sky {color}?', separator=' FUNKY TOWN ')
additional_template = StringTemplate('Why is the grass {color}?')
print((base_template + additional_template).render({'color': 'blue'}))

Why is the sky blue? FUNKY TOWN Why is the grass blue?


### 🧠 Level 3 - Use Links (lol)

What are Links? In this context, they represent atomic components of LLM interaction. That means they should be able to be strung together to form a Chain, although they can be used independently as well.

In [10]:
# The simplest link is the Link class, which just takes a prompt template and a model
link = Link(template=prompt_template, llm=model)
print(link)

Link: <['color']> -> <state>


In [11]:
# Invoke it using the input_dict, NOT keyword arguments
link(input_dict={'color': 'blue'}) # Or, equivalently, link.run(input_dict={'color': 'blue'})

{'state': "The color of the sky is primarily due to a phenomenon known as Rayleigh scattering. When sunlight enters Earth's atmosphere, it is composed of different colors, each characterized by different wavelengths. As this light passes through the atmosphere, it interacts with air molecules and other small particles.\n\nShorter wavelengths of light (blue and violet) are scattered in all directions by the gases and particles in the atmosphere much more than the longer wavelengths (red, yellow). Even though violet light is scattered even more than blue light, our eyes are more sensitive to blue light, and a lot of the violet light is absorbed by the upper atmosphere. So, what we predominantly see is blue light from all directions, giving the sky its characteristic color.\n\nIn summary, the blue color of the sky is mainly due to the scattering effect of sunlight by the particles and gases in the Earth's atmosphere, a phenomenon well-explained by Rayleigh scattering."}

In [12]:
# By default, output is stored under 'state'. You can get a copy (e.g., for inspection of intermediate results) by passing a different output key
link = Link(template=prompt_template, llm=model, output_key='intermediate')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue primarily due to Rayleigh scattering. Here's a more detailed explanation:\n\n1. **Nature of Sunlight**: Sunlight, or white light, is composed of many colors, each corresponding to a different wavelength. The visible spectrum ranges from violet (shortest wavelength) to red (longest wavelength).\n\n2. **Interaction with Atmosphere**: As sunlight passes through the Earth's atmosphere, it interacts with molecules and small particles in the air. This interaction preferentially scatters shorter wavelengths of light (violet and blue) more than longer wavelengths (red, yellow).\n\n3. **Rayleigh Scattering**: This scattering process, named after the British scientist Lord Rayleigh, describes how light particles are deflected more efficiently when they collide with particles much smaller than the wavelength of the light itself. Since blue light has a shorter wavelength compared to other colors in the visible spectrum, it is scattered in all directions by the gases

In [13]:
# You can name your links for easier debugging in logs
link = Link(template=prompt_template, llm=model, name='sky')
link.run(input_dict={'color': 'blue'})

{'state': "The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters Earth's atmosphere, it interacts with the gases and particles in the air. Sunlight consists of multiple colors, each with different wavelengths. The shorter wavelengths, such as blue and violet, are scattered more effectively by the molecules in the atmosphere compared to the longer wavelengths like red and yellow.\n\nAlthough violet light is scattered even more than blue light, our eyes are more sensitive to blue light and the upper atmosphere also absorbs some violet light. As a result, we perceive the sky as blue.\n\nThis scattering effect is more pronounced when the sun is higher in the sky, which is why the sky appears most blue during the middle of the day. At sunrise and sunset, sunlight has to pass through a larger portion of the atmosphere, scattering out most of the blue and violet light and leaving the longer wavelengths like red and orange, which is why we see those colo

In [14]:
# You can also build TransformationLinks, which take a function that transforms any input_dict into a specific output
def concatenate(input: dict):
    return f"{input['X']} and {input['Y']}"

concat_link = TransformationLink(
    input_keys=["X", "Y"], function=concatenate, output_key="C"
)

concat_link.run(input_dict={"X": "Hello", "Y": "World"})

{'state': 'Hello and World', 'C': 'Hello and World'}

### 🚀 Level 4 - Use Chains (lol again)

In [15]:
# You can also build Chains, which are just a sequence of links. There are two types of chains: LinearChain and Chain. 
# LinearChain is a sequence of links, while Chain is a collection of links that are invoked in the right order based on the input and output keys for each Link.

# This is a LinearChain #
#########################

# Build some links
link_opposite = Link(
    template=StringTemplate(
        "What's the opposite of {A}? Reply with a few words max."
    ),
    name = 'opposite',
    output_key="X",
)
link_superlative = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word."
    ),
    name='superlative',
    output_key="Y",
)
link_poem = Link(
    template=StringTemplate(
        "Write a poem about {state}. Return only the poem, beginning with the title."
    ),
    name='poem',
    output_key="Z",
)

# Now build a LinearChain
linear_chain = LinearChain(llm=model, links=[link_opposite, link_superlative, concat_link, link_poem])

# Run it
result = linear_chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})

# Get the output
print(result['state'])

[2024-07-09 17:55:45] [INFO] ℹ️ [chains.run:257] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X'] (added: ['state', 'X', 'llm'], removed: [])
[2024-07-09 17:55:46] [INFO] ℹ️ [chains.run:257] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y'] (added: ['Y'], removed: [])
[2024-07-09 17:55:46] [INFO] ℹ️ [chains.run:257] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y', 'C'] (added: ['C'], removed: [])
[2024-07-09 17:55:53] [INFO] ℹ️ [chains.run:257] 🔑 State keys: ['A', 'B', 'llm', 'state', 'X', 'Y', 'C', 'Z'] (added: ['Z'], removed: [])
**Outlaw's Sunshine**

Through shadowed veils and broken chains,
Where justice bends and freedom strains,
An outlaw strides with daring grin,
Defying fate, yet bound to win.

Beneath a sky of endless hues,
He treads on paths the law eschews,
Not merely rogue, but dreamer bright,
He seeks a world bathed in new light.

With every heist, and every scheme,
He gambles yet on one vast dream,
To topple walls that cage the soul,
To claim a life that's wild and whole

In [16]:
# A LinearChain will take the presence of the 'state' key into account when invoking the next link.
# A Chain will not. However, it has an extra 'feature' in the form of topological sorting. As long as a graph of links can be
# extracted from the chain, and the input keys (that are not generated in the chain) are given, the chain will invoke the links in the right order.

# This is a Chain #
###################
link_a = Link(
    template=StringTemplate(
        "Capitalize this word: {A}. Only respond with the capitalized version",
    ),
    name = 'opposite',
    output_key="X",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {B}. Reply with one word.", 
    ),
    name='superlative',
    output_key="Y",
)
link_c = Link(
    template=StringTemplate(
        "Combine {X} and {Y} and {C} in a meaningful sentence.", 
    ),
    name='sentence',
    output_key="Z",
)
chain = Chain(llm=model, links=[link_c, link_a, link_b]) # Note that we can pass the links in any order

# This is the sorted graph of links
chain._topological_sort()

[Link: superlative <['B']> -> <Y>,
 Link: opposite <['A']> -> <X>,
 Link: sentence <['Y', 'X', 'C']> -> <Z>]

In [17]:
# Now we can run it
try:
    result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful"})
except Exception as e:
    print(e)

The following input keys are required: {'C', 'B', 'A'}. Missing: {'C'}


In [18]:
# Woops, forgot something! There's no link that generates the 'C' key. We can pass it in though.
result = chain.run(input_dict={"A": "A police officer", "B": "Hopeful", "C": "a dog"})
print(result['state'])

The police officer, always optimistic about the future, trained his loyal dog to help in community outreach programs.


In [19]:
# Cycles are not allowed
link_up = Link(
    template=StringTemplate(
        "Capitalize this word: {up}. Only respond with the capitalized version",
    ),
    name = 'up',
    output_key="down",
)
link_b = Link(
    template=StringTemplate(
        "What's a synonym for {down}. Reply with one word.", 
    ),
    name='down',
    output_key="up",
)
chain = Chain(llm=model, links=[link_up, link_b])
try:
    chain._topological_sort()
except Exception as e:
    print(e)

Cycle detected in the dependency graph.
